# Import Librairies

## General

In [1]:
import numpy as np
import pandas as pd

In [2]:
from pycountry import countries
from datetime import datetime

In [3]:
from tqdm import tqdm
tqdm.pandas()

/Users/arthurmln/.pyenv/versions/3.7.6/envs/vivadata/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
def print_countries():
    for index in range(len(countries)):
        print(list(countries)[index])
        
print_countries()

Country(alpha_2='AW', alpha_3='ABW', name='Aruba', numeric='533')
Country(alpha_2='AF', alpha_3='AFG', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan')
Country(alpha_2='AO', alpha_3='AGO', name='Angola', numeric='024', official_name='Republic of Angola')
Country(alpha_2='AI', alpha_3='AIA', name='Anguilla', numeric='660')
Country(alpha_2='AX', alpha_3='ALA', name='Åland Islands', numeric='248')
Country(alpha_2='AL', alpha_3='ALB', name='Albania', numeric='008', official_name='Republic of Albania')
Country(alpha_2='AD', alpha_3='AND', name='Andorra', numeric='020', official_name='Principality of Andorra')
Country(alpha_2='AE', alpha_3='ARE', name='United Arab Emirates', numeric='784')
Country(alpha_2='AR', alpha_3='ARG', name='Argentina', numeric='032', official_name='Argentine Republic')
Country(alpha_2='AM', alpha_3='ARM', name='Armenia', numeric='051', official_name='Republic of Armenia')
Country(alpha_2='AS', alpha_3='ASM', name='American Samoa', n

# Import Datas

In [5]:
filepath = "../data/data_airplane_crash_20200623.csv"

import pickle
with open(filepath, 'rb') as f:
    df = pickle.load(f)

# Data Overview

In [6]:
df.head(3)

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,...,Summary,year,month,day,week_day_iso,hours,hour,AM_PM,prep1_country,alpha2_country
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,...,"During a demonstration flight, a U.S. Army fly...",1908,9,17,4,1900-01-01 17:18:00,17.0,PM,virginia,US
1,1909-09-07,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1.0,...,Eugene Lefebvre was the first pilot to ever be...,1909,9,7,2,NaT,NaN,None,france,FR
2,1912-07-12,06:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,...,First U.S. dirigible Akron exploded just offsh...,1912,7,12,5,1900-01-01 06:30:00,6.0,AM,new jersey,US


In [ ]:
df.shape

In [ ]:
df.info()

# Data Preparation

## Work on "Date" and "Time"

In [ ]:
def process_date(x):
    x = datetime.strptime(x, "%m/%d/%Y")
    return x

In [ ]:
def process_time(x):
    if type(x) == type("string"):
        x = datetime.strptime(x, "%H:%M")
        return x
    else:
        return x

In [ ]:
def am_pm(x):
    if x == "nan":
        return x
    
    else:
        if x > 12:
            return "PM"
        if x <= 12:
            return "AM"

In [ ]:
df.loc[158, "Time"] = "19:05"
df.loc[2125, "Time"] = "09:00"
df.loc[3100, "Time"] = "00:05"

In [ ]:
df["Date"] = df["Date"].progress_apply(process_date)

df["year"] = df["Date"].progress_apply(lambda x: x.year)
df["month"] = df["Date"].progress_apply(lambda x: x.month)
df["day"] = df["Date"].progress_apply(lambda x: x.day)
df["week_day_iso"] = df["Date"].progress_apply(lambda x: x.isoweekday())

#df_ok_time = df.loc[~df["Time"].isna()]

df["hours"] = df["Time"].progress_apply(process_time)
df["hour"] = df["hours"].progress_apply(lambda x: x.hour)
df["AM_PM"] = df["hour"].progress_apply(am_pm)


In [ ]:
df.head(3)

## Work on "Location" ==> Get Countries

In [ ]:
df["Location"].value_counts()

In [ ]:
stop_words_location = ['and', 'between', 'near', 'northern', 'of', 'off', 'over', 'the', 'western']

In [ ]:
def preprocessing_location_1(x):
    x = str(x).lower()
    split_loc = x.split(",")
    country = split_loc[-1]   
    
    liste_empty = list()
    for elt in country.split(" "):
        if elt not in stop_words_location:
            liste_empty.append(elt)
    
    return " ".join(liste_empty)

In [ ]:
def preprocessing_location_2(x):
    
    try:
        countries.search_fuzzy(x)[0].alpha_2
        #print(countries.search_fuzzy(x)[0].name)
        
    except:
        #print("-"*50)
        return "-"
    
    return countries.search_fuzzy(x)[0].alpha_2


In [ ]:
df["prep1_country"] = df["Location"].progress_apply(preprocessing_location_1)

In [ ]:
%%time
df["alpha2_country"] = df["prep1_country"].progress_apply(preprocessing_location_2)

### Dealing with elements where alpha_2 is Nan

In [9]:
df_alpha_emp = df.loc[df["alpha2_country"] == "-"]
df_alpha_emp

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,...,Summary,year,month,day,week_day_iso,hours,hour,AM_PM,prep1_country,alpha2_country
4,1913-09-09,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,...,The airship flew into a thunderstorm and encou...,1913,9,9,2,1900-01-01 18:30:00,18.0,PM,north sea,-
8,1916-07-28,NaN,"Near Jambol, Bulgeria",Military - German Army,NaN,NaN,Schutte-Lanz S-L-10 (airship),NaN,NaN,20.0,...,"Crashed near the Black Sea, cause unknown.",1916,7,28,5,NaT,NaN,None,bulgeria,-
15,1917-05-14,05:15,"Near Texel Island, North Sea",Military - German Navy,NaN,NaN,Zeppelin L-22 (airship),NaN,NaN,21.0,...,"Crashed into the sea from an altitude of 3,000...",1917,5,14,1,1900-01-01 05:15:00,5.0,AM,north sea,-
16,1917-06-14,08:45,"Off Vlieland Island, North Sea",Military - German Navy,NaN,NaN,Zeppelin L-43 (airship),NaN,NaN,24.0,...,Shot down by British aircraft.,1917,6,14,4,1900-01-01 08:45:00,8.0,AM,north sea,-
20,1918-04-07,21:30,Over the Mediterranean,Military - German Navy,NaN,NaN,Zeppelin L-59 (airship),NaN,NaN,23.0,...,Exploded and crashed into the sea off the sout...,1918,4,7,7,1900-01-01 21:30:00,21.0,PM,mediterranean,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4919,2016-08-31,11:00,"Russian Mission, Alaksa",Hageland Aviiation,3190,\tRussian Mission- Marshall,Cessna 208 Grand Caravan,\tN752RV,208B5088,5.0,...,A midair collision occurred between a Cessna 2...,2016,8,31,3,1900-01-01 11:00:00,11.0,AM,alaksa,-
4920,2016-10-24,07:20,Malta International Airport,AE Aviation,NaN,Malta-Luqa - Malta-Luga,Swearingen SA277-AT Expditer,N577MX,AT-577,5.0,...,"Shortly after taking off, the plane banked to ...",2016,10,24,1,1900-01-01 07:20:00,7.0,AM,malta international airport,-
4935,2017-07-10,16:00,"Near Itta Bena, Mississipi",USMC,NaN,"Cherry Point, NC - El Centro, CA",Lockheed KC-130T Hercules,165000,382-5303,16.0,...,"While en route, the Marine Corps transport suf...",2017,7,10,1,1900-01-01 16:00:00,16.0,PM,mississipi,-
4938,2017-10-14,08:30,"Abidjan, Ivory Coast",Valan International Cargo Charter,NaN,Burkina Faso - Abidjan,Antonov AN-26-100,ER-AVB,3204,10.0,...,"The cargo plane, carrying French military carg...",2017,10,14,6,1900-01-01 08:30:00,8.0,AM,ivory coast,-


In [8]:
df_alpha_emp["prep1_country"].value_counts()

 ussr                         67
 south vietnam                40
 laos                         21
 south korea                  16
 democratic republic congo    15
                              ..
 taiwan (formosa)              1
petreasa romania               1
 south dekota                  1
northeast laos                 1
 east timor                    1
Name: prep1_country, Length: 216, dtype: int64

In [ ]:
def alpha2_hand(df):
    df_alpha_emp = df.loc[df["alpha2_country"] == "-"]
    words_not_understood = df_alpha_emp["prep1_country"].value_counts().index
    
    for elt in words_not_understood:
        
        to_print_0 = "elif x == "+"'"+elt+"'"+":"        
        
        to_print_1 = "    "+"index_df = df_alpha_emp.loc[df_alpha_emp['prep1_country'] == "
        to_print_2 = "'"
        to_print_3 = elt
        to_print_4 = "']"
        to_print_5 = ".index"
        to_print_f = to_print_1+to_print_2+to_print_3+to_print_4+to_print_5
        
        to_print_11 = "    "+"for index in index_df:"
        to_print_12 = "    "+"    "+"df_alpha_emp.loc[index, 'alpha2_country'] = '"
        
        
        print(to_print_0)
        print(to_print_f)
        print(to_print_11)
        print(to_print_12)
        print()
       
   

In [ ]:
def alpha2_hand(df):
    df_alpha_emp = df.loc[df["alpha2_country"] == "-"]
    words_not_understood = df_alpha_emp["prep1_country"].value_counts().index
    
    for elt in words_not_understood:
        
        to_print_0 = "    "+"elif x == "+"'"+elt+"'"+":"         
        to_print_1 = "    "+"    "+"return"+" "+"''"
        
        print(to_print_0)
        print(to_print_1)
        print()
       
   

In [10]:
def alpha2_nan_preprocessing(x):
    if x == ' ussr':
        return 'RU'

    elif x == ' south vietnam':
        return 'VN'

    elif x == ' laos':
        return 'LA'

    elif x == ' south korea':
        return 'KR'

    elif x == ' democratic republic congo':
        return 'CD'

    elif x == ' czechoslovakia':
        return 'SK'

    elif x == ' yugoslavia':
        return 'RS'

    elif x == ' burma':
        return 'MM'

    elif x == ' west germany':
        return 'DE'

    elif x == 'atlantic ocean':
        return 'ATLANTIC OCEAN'

    elif x == 'north atlantic ocean':
        return 'ATLANTIC OCEAN'

    elif x == 'english channel':
        return 'ENGLISH CHANNEL SEA'

    elif x == 'mediterranean sea':
        return 'MED SEA'

    elif x == 'pacific ocean':
        return 'PACIFIC OCEAN'

    elif x == 'north pacific ocean':
        return 'PACIFIC OCEAN'

    elif x == ' ivory coast':
        return 'CI'

    elif x == ' north sea':
        return 'NORTH SEA'

    elif x == 'north sea':
        return 'NORTH SEA'

    elif x == ' canary islands':
        return 'ES'

    elif x == ' netherlands antilles':
        return 'VE'

    elif x == ' tennesee':
        return 'US'

    elif x == ' azores':
        return 'PT'

    elif x == ' west indies':
        return 'BS'

    elif x == 'algiers':
        return 'DZ'

    elif x == ' pacific ocean':
        return 'PACIFIC OCEAN'

    elif x == ' east germany':
        return 'DE'

    elif x == ' atlantic ocean':
        return 'ATLANTIC OCEAN'

    elif x == 'gulf mexico':
        return 'MX'

    elif x == 'gulf finland':
        return 'FI'

    elif x == 'north atlantic':
        return 'ATLANTIC OCEAN'

    elif x == ' comoro islands':
        return 'KM'

    elif x == ' amsterdam':
        return 'NL'

    elif x == ' minnisota':
        return 'US'

    elif x == 'south atlantic ocean':
        return 'ATLANTIC OCEAN'

    elif x == ' crete':
        return 'GR'

    elif x == ' north korea':
        return 'KP'

    elif x == ' french west africa':
        return 'ML'

    elif x == ' west pakistan':
        return 'PK'

    elif x == ' belgian congo':
        return 'CD'

    elif x == ' alaksa':
        return 'US'

    elif x == ' french indo-china':
        return 'VN'

    elif x == ' dr congo':
        return 'CD'

    elif x == ' wisconson':
        return 'US'

    elif x == 'gulf tonkin':
        return 'VN'

    elif x == ' chechnya':
        return 'RU'

    elif x == ' d.c.':
        return 'US'

    elif x == ' french equatorial africa':
        return 'TD'

    elif x == ' saudia arabia':
        return 'SA'

    elif x == ' democtratic republic congo':
        return 'CD'

    elif x == 'fox glacier airstrip':
        return 'NZ'

    elif x == ' french west indies':
        return 'BS'

    elif x == 'territory new guinea':
        return 'PG'

    elif x == 'himalayas':
        return 'NP'

    elif x == 'midway island naval air station':
        return 'US'

    elif x == ' kirghizia':
        return 'KG'

    elif x == ' jamacia':
        return 'JM'

    elif x == ' sierre leone':
        return 'SL'

    elif x == ' uzbekstan':
        return 'UZ'

    elif x == ' forest-in-teesdale. united kingdom':
        return 'GB'

    elif x == ' coloado':
        return 'US'

    elif x == 'centeral afghanistan afghanistan':
        return 'AF'

    elif x == ': massachusetts':
        return 'US'

    elif x == ' saint lucia island':
        return 'LC'

    elif x == ' south dekota':
        return 'US'

    elif x == 'en route from argentina  to  california':
        return 'AR'

    elif x == '  canary islands':
        return 'ES'

    elif x == ' yugosalvia':
        return 'RS'

    elif x == 'aegean sea':
        return 'AEGEAN SEA'

    elif x == ' new york (idlewild)':
        return 'US'

    elif x == ' baangladesh':
        return 'BD'

    elif x == ' black sea':
        return 'BLACK SEA'

    elif x == 'karkov':
        return 'UA'

    elif x == ' rhodesia (zimbabwe)':
        return 'ZW'

    elif x == 'malta international airport':
        return 'MT'

    elif x == '  pest hungary':
        return 'HU'

    elif x == ' khmer republic':
        return 'KH'

    elif x == 'coast france':
        return 'FR'

    elif x == 'irish coast':
        return 'IE'

    elif x == ' washingon':
        return 'US'

    elif x == 'chili':
        return 'CL'

    elif x == ' nambia':
        return 'NA'

    elif x == 'glasgow scotland':
        return 'GB'

    elif x == ' philippine island elalat':
        return 'PH'

    elif x == ' mauretania':
        return 'MR'

    elif x == '175 miles egyptian coast':
        return 'EG'

    elif x == ' south kazakistan':
        return 'KZ'

    elif x == ' hunary':
        return 'HU'

    elif x == 'irkutsk russia':
        return 'RU'

    elif x == 'central mozambique':
        return 'MZ'

    elif x == ' u.s. samoa':
        return 'US'

    elif x == ' bosnia-herzegovina':
        return 'BA'

    elif x == ' inodnesia':
        return 'ID'

    elif x == ' netherlands indies':
        return 'NE'

    elif x == ' taiwan (formosa)':
        return 'TW'

    elif x == 'eugene island':
        return 'US'

    elif x == ' borneo':
        return 'MY'

    elif x == ' unied kingdom':
        return 'GB'

    elif x == 'alaska coast':
        return 'US'

    elif x == ' cameroons':
        return 'CM'

    elif x == ' 325 miles east wake island':
        return 'ATLANTIC OCEAN'

    elif x == ' northwest territories canada':
        return 'CA'

    elif x == ' deleware':
        return 'US'

    elif x == 'sao tomé & principe':
        return 'ST'

    elif x == '  democratic republic congo':
        return 'CD'

    elif x == ' malagasy republic':
        return 'MG'

    elif x == ' mississipi':
        return 'US'

    elif x == 'chicago illinois':
        return 'US'

    elif x == 'south indian ocean':
        return 'INDIAN OCEAN'

    elif x == ' zimbabwe)':
        return 'ZW'

    elif x == ' upper volta':
        return 'BF'

    elif x == ' east pakistan':
        return 'PK'

    elif x == '  atlantic ocean':
        return 'ATLANTIC OCEAN'

    elif x == ' india.':
        return 'IN'

    elif x == 'panama coast':
        return 'PA'

    elif x == ' burma (myanmar)':
        return 'MM'

    elif x == ' azores (portugal)':
        return 'PT'

    elif x == ' cailifornia':
        return 'US'

    elif x == ' djbouti':
        return 'DJ'

    elif x == 'south gibraltar':
        return 'GI'

    elif x == ' democratic republic cogo':
        return 'CD'

    elif x == ' persian gulf':
        return 'IR'

    elif x == ' ilinois':
        return 'US'

    elif x == ' 110 miles west ireland':
        return 'ATLANTIC OCEAN'

    elif x == ' aregntina':
        return 'AR'

    elif x == ' congo democratic republic':
        return 'CD'

    elif x == 'turks caicos islands':
        return 'TR'

    elif x == ' tanganyika':
        return 'TZ'

    elif x == 'mediterranean':
        return 'MED SEA'

    elif x == 'tachikawa air base':
        return 'JP'

    elif x == ' washington dc':
        return 'US'

    elif x == ' bugaria':
        return 'BG'

    elif x == ' napal':
        return 'NP'

    elif x == ' french cameroons':
        return 'CM'

    elif x == ' virginia.':
        return 'US'

    elif x == 'petreasa romania':
        return 'RO'

    elif x == ' east timor':
        return 'TL'

    elif x == 'atlantic ocean florida':
        return 'ATLANTIC OCEAN'

    elif x == ' calilfornia':
        return 'US'

    elif x == ' us virgin islands':
        return 'VI'

    elif x == ' south yemen':
        return 'YE'

    elif x == 'gulf thailand':
        return 'TH'

    elif x == 'coatia':
        return 'HR'

    elif x == ' comoros islands':
        return 'KM'

    elif x == 'hong kong international airport':
        return 'HK'

    elif x == ' swden':
        return 'SE'

    elif x == ' australila':
        return 'AU'

    elif x == ' domincan republic':
        return 'DO'

    elif x == ' republic djibouti':
        return 'DJ'

    elif x == ' morroco':
        return 'MA'

    elif x == ' cape verde islands':
        return 'CV'

    elif x == ' british columbia canada':
        return 'CA'

    elif x == ' alakska':
        return 'US'

    elif x == ' u.s. virgin islands':
        return 'VI'

    elif x == 'northeast laos':
        return 'LA'

    elif x == ' tahiti':
        return 'PACIFIC OCEAN'

    elif x == ' bulgeria':
        return 'BG'

    elif x == 'south vietnam':
        return 'VN'

    elif x == ' spain canary islands':
        return 'ES'

    elif x == 'west africa':
        return 'ML'

    elif x == ' afghanstan':
        return 'AF'

    elif x == ' gulf karkinitsky':
        return 'UA'

    elif x == ' belgium congo':
        return 'CD'

    elif x == 'isle man':
        return 'GB'

    elif x == 'são tomé island':
        return 'ST'

    elif x == ' argenina':
        return 'AR'

    elif x == 'pacific ocean hong kong macao':
        return 'HK'

    elif x == ' queensland  australia':
        return 'AU'

    elif x == ' philipines':
        return 'PH'

    elif x == ' french somaliland':
        return 'DJ'

    elif x == ' rhodesia':
        return 'ZW'

    elif x == 'mt. helmos. greece':
        return 'GR'

    elif x == ' hi)':
        return 'PACIFIC OCEAN'

    elif x == 'malta-luqa':
        return 'MT'

    elif x == 'eastern libya':
        return 'LY'

    elif x == 'gulf sirte':
        return 'MED SEA'

    elif x == 'milford sound':
        return 'NZ'

    elif x == 'pacific ocean manila guam':
        return 'PACIFIC OCEAN'

    elif x == ' canada2':
        return 'CA'

    elif x == 'baltic sea':
        return 'BALTIC SEA'

    elif x == ' belgian congo (zaire)':
        return 'CD'

    elif x == 'jacquinot bay  new guinea':
        return 'PG'

    elif x == ' qld. australia':
        return 'AU'

    elif x == 'villia greece':
        return 'GR'

    elif x == 'barquisimeto venezuela':
        return 'VE'

    elif x == 'oregon coast':
        return 'US'

    elif x == 'washington d.c.':
        return 'US'

    elif x == ' phillipines':
        return 'PH'

    elif x == 'gulf oman':
        return 'OM'

    elif x == ' south-west  africa (namibia)':
        return 'NA'

    elif x == 'southeastern bolivia':
        return 'BO'

    elif x == ' uae':
        return 'AE'

    elif x == ' massachutes':
        return 'US'

    elif x == ' dutch guyana':
        return 'SR'

    elif x == 'sea japan':
        return 'JP'

    elif x == ' islay island':
        return 'GB'

    elif x == ' ne bermuda':
        return 'BM'

    elif x == 'tabones island  philippines':
        return 'PH'

    elif x == 'persian gulf':
        return 'IR'

    elif x == ' great inagua':
        return 'BS'

    elif x == 'houma louisiana':
        return 'US'

    elif x == ' cape verde':
        return 'CV'

    elif x == 'florida coast':
        return 'US'

    elif x == ' south korean':
        return 'KR'

    elif x == ' leeward islands':
        return 'VI'

    elif x == ' airzona':
        return 'US'

    elif x == ' east sardinia':
        return 'IT'

    elif x == ' guizhou province':
        return 'CN'

    elif x == ' 800 miles east newfoundland':
        return 'ATLANTIC OCEAN'

    elif x == ' philippine sea':
        return 'PH'

    elif x == 'java sea':
        return 'ID'

    elif x == ' quebec canada':
        return 'CA'

    elif x == 'atlantic ocean n.y. bermuda':
        return 'BM'

    elif x == 'andaman sea':
        return 'INDIAN OCEAN'

    elif x == ' soviet union':
        return 'RU'

    elif x == ' republic georgia':
        return 'GE'

    elif x == 'sao gabriel de cachoeria':
        return 'BR'
    

### Merge DF_Alphas

In [11]:
df_alpha_emp = df.loc[df["alpha2_country"] == "-"]
df_alpha_ok = df.loc[df["alpha2_country"] != "-"]

In [12]:
len(df_alpha_emp) + len(df_alpha_ok) == len(df)

True

In [13]:
df_alpha_emp["alpha2_country"] = df_alpha_emp["prep1_country"].progress_apply(alpha2_nan_preprocessing)

100%|██████████| 501/501 [00:00<00:00, 197476.39it/s]
/Users/arthurmln/.pyenv/versions/3.7.6/envs/vivadata/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df = pd.concat([df_alpha_emp, df_alpha_ok], axis=0).sort_index()

### Get GPS Coordinates for each country

In [16]:
filepath_countries = "../data/countries_codes_and_coordinates.csv"
df_countries = pd.read_csv(filepath_countries)

In [17]:
df_countries["Latitude (average)"] = df_countries["Latitude (average)"].apply(lambda x: float(x[2:-1]))
df_countries["Longitude (average)"] = df_countries["Longitude (average)"].apply(lambda x: float(x[2:-1]))
df_countries["Alpha-2 code"] = df_countries["Alpha-2 code"].apply(lambda x: str(x[2:-1]))

df_countries = df_countries.drop([25, 32, 55, 118, 127, 154, 186, 194, 195, 221, 245, 247])
df_countries = df_countries.reset_index()
df_countries.drop("index", axis=1, inplace=True)

In [19]:
alpha_2_ocean = df.loc[df["alpha2_country"].str.contains("OCEAN")]["alpha2_country"].value_counts()
alpha_2_sea = df.loc[df["alpha2_country"].str.contains("SEA")]["alpha2_country"].value_counts()

print(alpha_2_ocean.index)
print(alpha_2_sea.index)

Index(['ATLANTIC OCEAN', 'PACIFIC OCEAN', 'INDIAN OCEAN'], dtype='object')
Index(['NORTH SEA', 'MED SEA', 'ENGLISH CHANNEL SEA', 'AEGEAN SEA',
       'BLACK SEA', 'BALTIC SEA'],
      dtype='object')


In [20]:
df_water = pd.DataFrame(data=[['ATLANTIC OCEAN', 'ATLANTIC OCEAN', "--", "--", 26, 42],
                              ['PACIFIC OCEAN', 'PACIFIC OCEAN', "--", "--", -10, -140],
                              ['INDIAN OCEAN', 'INDIAN OCEAN', "--", "--", -25, 100],
                              ['NORTH SEA', 'NORTH SEA', "--", "--", 54, 13],
                              ['MED SEA', 'MED SEA', "--", "--", 42, 14],
                              ['ENGLISH CHANNEL SEA', 'ENGLISH CHANNEL SEA', "--", "--", 46, 6],
                              ['BALTIC SEA', 'BALTIC SEA', "--", "--", 55, 15],
                              ['AEGEAN SEA', 'AEGEAN SEA', "--", "--", 39, 25],
                              ['BLACK SEA', 'BLACK SEA', "--", "--", 37, 16]],
                        index = np.arange(len(df_countries), len(df_countries)+9),
                        columns = df_countries.columns)

In [21]:
df_gps = pd.concat([df_countries, df_water], axis=0)

In [31]:
def get_gps_lat(x):
    #try :
    lat = float(df_gps.loc[df_gps["Alpha-2 code"] == str(x)]["Latitude (average)"].values)
        #print(lat)
    return lat
   #except :
        #print(f"{x} not found")
        #print("-"*50)
        #print("-"*50)
        #print("-"*50)

In [32]:
def get_gps_long(x):
    #try :
    long = float(df_gps.loc[df_gps["Alpha-2 code"] == str(x)]["Longitude (average)"].values)
        #print(long)
    return long
    #except :
        #print(f"{x} not found")
        #print("-"*50)
        #print("-"*50)
        #print("-"*50)

In [34]:
df["latitude"] = df["alpha2_country"].progress_apply(get_gps_lat)
df["longitude"] = df["alpha2_country"].progress_apply(get_gps_long)
df.head(3)

100%|██████████| 4967/4967 [00:02<00:00, 2144.59it/s]


,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,...,month,day,week_day_iso,hours,hour,AM_PM,prep1_country,alpha2_country,latitude,longitude
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,...,9,17,4,1900-01-01 17:18:00,17.0,PM,virginia,US,38.0,-97.0
1,1909-09-07,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1.0,...,9,7,2,NaT,NaN,None,france,FR,46.0,2.0
2,1912-07-12,06:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,...,7,12,5,1900-01-01 06:30:00,6.0,AM,new jersey,US,38.0,-97.0


# Save Data

In [35]:
import pickle
with open("../data/data_airplane_crash_20200624.pkl", "wb") as f:
    pickle.dump(df, f)   